<a href="https://colab.research.google.com/github/BimbaBuk/Personal-Projects-/blob/main/NN_shared_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from google.colab import drive  # Import for Google Drive mounting
drive.mount('/content/drive')  # Mount Google Drive
drive_dir = '/content/drive/My Drive/NN_Models_Shared/' # Our google Drive directory

In [ ]:
def train_model(df, input_columns, target_columns, model_name, MLP_structure, num_epochs, batch_size, learning_rate):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    full_name = model_name + '_MLP_' + MLP_structure + '_epochs_' + str(num_epochs) + '_batch_' + str(batch_size) + '_lr_' + str(learning_rate)
    model_dir = drive_dir + full_name + '/'

    input_data = df[input_columns].values
    target_data = df[target_columns].values
    input_data_scaled = StandardScaler().fit_transform(input_data)
    target_data_scaled = StandardScaler().fit_transform(target_data)

    # Split the data into train and validation sets
    X_train, X_test, y_train, y_test = train_test_split(input_data_scaled, target_data_scaled, test_size=0.2, random_state=42)

    # Convert data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

    # Create datasets and dataloaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # Initialize model, loss function, and optimizer
    model = MLP(input_size=len(input_columns), output_size=len(target_columns)).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    writer = SummaryWriter(log_dir=model_dir) # TensorBoard setup

    # Training loop
    for epoch in range(num_epochs):
        model.train() # set the model to training mode
        train_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            predictions = model(inputs)
            loss = criterion(predictions, targets)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                predictions = model(inputs)
                loss = criterion(predictions, targets)
                val_loss += loss.item()

        # Calculate average losses
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

        # Log losses to TensorBoard
        writer.add_scalar('Loss/Train', train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)

        # Add model graph to TensorBoard (only for the first epoch)
        if epoch == 0:
            dummy_input = torch.randn(1, len(input_columns)).to(device)
            writer.add_graph(model, dummy_input)

    writer.flush()  # Ensure all data is written to TensorBoard
    writer.close()

    # Save the model to Google Drive
    model_path = model_dir + model_name + '.pth'
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to Google Drive as {model_name}")
    return model_dir

In [ ]:
# Define the MLP model!!!!!!!!!!!!!!!!!!!!!!
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, output_size)
        )
    def forward(self, x):
        return self.model(x)

if __name__ == '__main__':
    # Load the csv from google drive
    csv_name = 'overfit_test.csv' # Choose here csv!!!!!!!!!!!!!!!!!!!!!!
    df = pd.read_csv(drive_dir + csv_name)

    # Define the input and target columns
    input_columns  = ['theta', 'velocity', 'yaw_angle', 'yaw_rate', 'slip_angle', 'front_wheel',
                      'back_wheel', 'acceleration', 'angular_acceleration']  # C to K
    target_columns = ['delta_x_position', 'delta_y_position', 'delta_theta', 'delta_velocity', 'delta_yaw_angle',
                      'delta_yaw_rate', 'delta_slip_angle', 'delta_front_wheel', 'delta_back_wheel']  # L to T

    # Train the model: returns the directory where the model saved
    model_dir = train_model(df, input_columns, target_columns, model_name="overfitcheck", MLP_structure='256x128', num_epochs=100, batch_size=128, learning_rate=0.001) # Modify!!!!!!!!!!!!!!!!!!!!!!

    # Show the training process on TensorBoard
    %load_ext tensorboard
    %tensorboard --logdir="$model_dir"

In [ ]:
    # Show a saved training process on TensorBoard
    model_name = 'overfitcheck_MLP_256x128_epochs_100_batch_128_lr_0.001' # change model name to an existing file in drive!!!!!!!!!!!!!!!!!!!!!!
    model_dir = drive_dir + model_name

    %load_ext tensorboard
    %tensorboard --logdir="$model_dir"





    מלכךדגףחכדףלF'S'